In [ ]:
import argparse
import collections
import gzip
import json
import math
import multiprocessing
import numpy as np
import os
import pickle
import random
import re
import six
import subprocess
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
import matplotlib

from multiprocessing import Lock, Manager, Pool, Process, Queue, Value
from six import iteritems
from six.moves import range as six_range

In [ ]:
!pip install transformers

from transformers import BertModel, BertTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/2020_09_12_topic_model_공유/data

In [ ]:
ls

In [ ]:
fp2 = open('nq/predict(정학).trec', 'r', encoding='utf-8')  #여기엔 bm25 
predict = []
while True:
    line = fp2.readline()
    if not line: break
    predict.append(line.split())
bm25_predict=sorted(predict , key = lambda x : (int(x[0]), int(x[3])))
print(bm25_predict[0])
fp2.close()

In [ ]:
!chmod 777 trec_eval

In [ ]:
cd /content/drive/My Drive/2020_09_16_combine/data/

In [ ]:
ls

In [ ]:
!chmod 777 trec_eval

In [ ]:
data = open('nq/lendata.txt', 'r', encoding='utf-8') #여기엔 내가 준 파일 넣기
lendocu = []
while True:
    line = data.readline()
    if not line: break
    lendocu.append(line)
temp = []
temp1 =[]
temp.append(lendocu[0].split())
for i in range(len(temp[0])):
  temp1.append(int(temp[0][i]))  
lendocu = temp1
print(lendocu[0])

In [ ]:
fp = open('nq/new_predict.trec', 'r', encoding='utf-8') #여기엔 bert predict
predict = []
while True:
    line = fp.readline()
    if not line: break
    predict.append(line.split())
bert_predict=sorted(predict , key = lambda x : (int(x[0]), int(x[3])))
print(bert_predict[0])
fp.close()

In [ ]:
bert_score = []
for i in range(0,633):
  for j in range(0,1000):
    k = i*1000+j
    bert_score.append(bert_predict[k][4])
    index = int(bm25_predict[k][2])
    bool = 0
    if lendocu[index] >300:
      for t in range((k-j),(k-j+1000)):
        if index == int(bert_predict[t][2]):
          bool =1
          break
      if bool ==1:
        if int(bert_predict[t][3]) > int(bm25_predict[k][3]):
          mid = int(int(bert_predict[t][3])*0.3+int(bm25_predict[k][3])*0.7)
          temp = bert_predict[t]
          del bert_predict[t]
          bert_predict.insert((k-j+mid)-1,temp)
        else:
          mid = int(int(bert_predict[t][3])*0.3+int(bm25_predict[k][3])*0.7)
          temp = bert_predict[t]
          del bert_predict[t]
          bert_predict.insert((k-j+mid)-1,temp)
    elif lendocu[index] >400:
      for t in range((k-j),(k-j+1000)):
        if index == int(bert_predict[t][2]):
          bool =1
          break
      if bool ==1:
        if int(bert_predict[t][3]) > int(bm25_predict[k][3]):
          mid = int(int(bert_predict[t][3])*0.2+int(bm25_predict[k][3])*0.8)
          temp = bert_predict[t]
          del bert_predict[t]
          bert_predict.insert((k-j+mid)-1,temp)
        else:
          mid = int(int(bert_predict[t][3])*0.2+int(bm25_predict[k][3])*0.8)
          temp = bert_predict[t]
          del bert_predict[t]
          bert_predict.insert((k-j+mid)-1,temp)
    elif lendocu[index] >500:
      for t in range((k-j),(k-j+1000)):
        if index == int(bert_predict[t][2]):
          bool =1
          break
      if bool ==1:
        if int(bert_predict[t][3]) > int(bm25_predict[k][3]):
          mid = int(int(bert_predict[t][3])*0.1+int(bm25_predict[k][3])*0.9)
          temp = bert_predict[t]
          del bert_predict[t]
          bert_predict.insert((k-j+mid)-1,temp)
        else:
          mid = int(int(bert_predict[t][3])*0.1+int(bm25_predict[k][3])*0.9)
          temp = bert_predict[t]
          del bert_predict[t]
          bert_predict.insert((k-j+mid)-1,temp)

In [ ]:
fw = open('nq/last_predict.trec', 'w', encoding='utf-8')
for i in range(0,633):
  for j in range(0,1000):
    k = i*1000+j
    bert_predict[k][3] = k%1000 +1
    bert_predict[k][4] = str(bert_score[k])
    fw.write('%s %s %s %d %s %s\n' % (bert_predict[k][0],bert_predict[k][1], bert_predict[k][2],bert_predict[k][3], bert_predict[k][4], bert_predict[k][5]))
fw.close()    

In [ ]:
trec_eval_res = subprocess.Popen(['./trec_eval', '-m', 'all_trec', 'nq/answer.trec', 'nq/new_predict.trec'], stdout=subprocess.PIPE, shell=False)
out, err = trec_eval_res.communicate()
lines = out.decode('utf-8').strip().split('\n')
metrics = {}
for line in lines[1:]:
    metric, _, value = line.split()
    if '.' in value:
        value = float(value)
    else:
        value = int(value)
    metrics[metric.lower()] = value
print('| R@1000 |  R@500  | R@200 |  R@100  | R@30 |')
print('| %6.2f | %6.2f | %6.2f | %6.2f | %6.2f |' % (metrics['recall_1000'] * 100, metrics['recall_500'] * 100, metrics['recall_200'] * 100, metrics['recall_100'] * 100, metrics['recall_30'] * 100 ))

In [ ]:
trec_eval_res = subprocess.Popen(['./trec_eval', '-m', 'all_trec', 'nq/answer.trec', 'nq/last_predict.trec'], stdout=subprocess.PIPE, shell=False)
out, err = trec_eval_res.communicate()
lines = out.decode('utf-8').strip().split('\n')
metrics = {}
for line in lines[1:]:
    metric, _, value = line.split()
    if '.' in value:
        value = float(value)
    else:
        value = int(value)
    metrics[metric.lower()] = value
print('| R@1000 |  R@500  | R@200 |  R@100  | R@30 |')
print('| %6.2f | %6.2f | %6.2f | %6.2f | %6.2f |' % (metrics['recall_1000'] * 100, metrics['recall_500'] * 100, metrics['recall_200'] * 100, metrics['recall_100'] * 100, metrics['recall_30'] * 100 ))

In [ ]:
fa_tok = open('nq/answer.txt', 'r', encoding='utf-8')
fv = open('nq/vocab_30000.txt', 'r', encoding='utf-8')

q = []
vocab = []
cnt = 0

for line in fa_tok:
    if cnt % 2 == 0:
        temp = line.split()
        q.append(temp)
        #query_tok.append(tokenizer.tokenize(q))
    cnt += 1
fa_tok.close()

for line in fv:
    temp = line.split()
    vocab.append(temp[1])
fv.close()

In [ ]:
for i in range(10):
    for s in q:
        for w in s:
            if w in vocab:
                s.remove(w)

In [ ]:
a = 0
b = 0
t_list = []
for i in q:
    if i == []:
        a += 1
    else:
        for w in i:
            if '\'' in w:
                i.remove(w)
            else:
                t_list.append(i)
                b += 1

In [ ]:
fd = open('nq/docset_tok.txt', 'r', encoding='utf-8')

doc_tok = []

for line in fd:
    doc_tok.append(line)
fd.close()

In [ ]:
rank_margin = 20
rank_ratio = 0.5

In [ ]:
fp = open('nq/predict.trec', 'r', encoding='utf-8')
out_fp = open('nq/new_predict.trec', 'w', encoding='utf-8')

qn = cnt//2
k = 0
for n in range(qn):
    p_list = []
    for i in range(1000):
        line = fp.readline()
        p_list.append(line.split())
    if len(q[n]) > 0:
        for w in q[n]:
            for p in p_list:
                did = int(p[2])
                if w in doc_tok[did]:
                    if int(p[3]) > rank_margin:
                        k += 1
                        p_list.remove(p)
                        r = int(p[3])
                        p[3] = str(int(r * rank_ratio) + 1)
                        p[4] = str(float(p_list[int(r * rank_ratio) - 1][4]) + float(p_list[int(r * rank_ratio)][4]) / 2)
                        p_list.insert(int(r * rank_ratio), p)
    i = 0
    for p in p_list:
        out_fp.write('%s %s %s %d %s %s\n' % (p[0], p[1], p[2], i + 1, p[4], p[5]))
        i += 1 
out_fp.close()
fp.close()

In [ ]:
trec_eval_res = subprocess.Popen(['./trec_eval', '-m', 'all_trec', 'nq/answer.trec', 'nq/new_predict.trec'], stdout=subprocess.PIPE, shell=False)
out, err = trec_eval_res.communicate()
lines = out.decode('utf-8').strip().split('\n')
metrics = {}
for line in lines[1:]:
    metric, _, value = line.split()
    if '.' in value:
        value = float(value)
    else:
        value = int(value)
    metrics[metric.lower()] = value
print('| R@1000 |  R@500  | R@200 |  R@100  | R@30 |')
print('| %6.2f | %6.2f | %6.2f | %6.2f | %6.2f |' % (metrics['recall_1000'] * 100, metrics['recall_500'] * 100, metrics['recall_200'] * 100, metrics['recall_100'] * 100, metrics['recall_30'] * 100 ))